In [63]:
from tensorflow.keras.models import load_model
import numpy as np
import pandas as pd
import os
import librosa
import librosa.display
import numpy as np
import IPython.display as ipd
from tqdm import tqdm
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline
import tensorflow_io as tfio
import math
import json

In [3]:
DATASET_PATH = os.path.join('ASR_Diplom', 'YANDEX_DATASET', 'to_learn', 'to_learn_2' )

In [9]:
model = load_model('saved_models/audio_classification.hdf5')

2023-06-11 10:19:14.689648: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-06-11 10:19:14.694230: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [58]:
extracted_features_df = pd.read_csv('my_data.csv')
extracted_features_df.head(10)

,feature,class
0,[-6.94500854e+02 8.17331696e+01 -4.69072952e+...,dislike_to_learn
1,[-7.6508453e+02 7.3037628e+01 -3.5624687e+01 ...,dislike_to_learn
2,[-770.0587 73.25779 -35.258 35.2...,dislike_to_learn
3,[-7.2832935e+02 7.1925896e+01 -2.1925377e+01 ...,dislike_to_learn
4,[-6.3760278e+02 7.3678200e+01 -3.8586246e+01 ...,dislike_to_learn
5,[-7.6653198e+02 6.9944092e+01 -2.6869017e+01 ...,dislike_to_learn
6,[-7.1310046e+02 4.4588623e+01 -1.4694318e+01 ...,dislike_to_learn
7,[-6.92225891e+02 4.21372032e+01 -3.43386536e+...,dislike_to_learn
8,[-7.3918774e+02 7.4243317e+01 -3.9925903e+01 ...,dislike_to_learn
9,[-6.8446661e+02 8.6852219e+01 -2.2097857e+01 ...,dislike_to_learn


In [59]:
x=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

In [60]:
x.shape

(4000,)

In [15]:
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
labelencoder=LabelEncoder()
y=to_categorical(labelencoder.fit_transform(y))

In [10]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 100)               4100      
                                                                 
 activation_8 (Activation)   (None, 100)               0         
                                                                 
 dropout_6 (Dropout)         (None, 100)               0         
                                                                 
 dense_9 (Dense)             (None, 200)               20200     
                                                                 
 activation_9 (Activation)   (None, 200)               0         
                                                                 
 dropout_7 (Dropout)         (None, 200)               0         
                                                                 
 dense_10 (Dense)            (None, 200)              

In [18]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

def preprocess(file_path): 
    wav = load_wav_16k_mono(file_path)
    wav = wav[:48000]
    zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    wav = tf.concat([zero_padding, wav],0)
    return wav

In [46]:
import subprocess
subprocess.call(['ffmpeg', '-i', 'flask_serv/saved/file.wav',
                   'flask_serv/saved/audio.wav'])

ffmpeg version n6.0 Copyright (c) 2000-2023 the FFmpeg developers
  built with gcc 13.1.1 (GCC) 20230429
  configuration: --prefix=/usr --disable-debug --disable-static --disable-stripping --enable-amf --enable-avisynth --enable-cuda-llvm --enable-lto --enable-fontconfig --enable-gmp --enable-gnutls --enable-gpl --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libdav1d --enable-libdrm --enable-libfreetype --enable-libfribidi --enable-libgsm --enable-libiec61883 --enable-libjack --enable-libjxl --enable-libmfx --enable-libmodplug --enable-libmp3lame --enable-libopencore_amrnb --enable-libopencore_amrwb --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librav1e --enable-librsvg --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtheora --enable-libv4l2 --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable

0

In [64]:
# file_name = os.path.join(DATASET_PATH, 'up_to_learn', '0b4ef445b92a18e374c8bf1423cadae8.wav')
file_name = 'flask_serv/saved/audio.wav'
audio_data, sample_rate = librosa.core.load(file_name)
mfccs_features = librosa.feature.mfcc(y=audio_data, sr=16000, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)

print(mfccs_scaled_features)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
print(mfccs_scaled_features)
print(mfccs_scaled_features.shape)
predicted_label=model.predict(mfccs_scaled_features)
print(predicted_label)
classes_x=np.argmax(predicted_label,axis=1)
prediction_class = labelencoder.inverse_transform(classes_x)
prediction_class[0]

[-6.8120978e+02  3.8078697e+01  2.9524112e-01  5.3980947e+00
  9.3655224e+00  2.9750962e+00  2.3212700e+00  2.7702498e+00
 -2.1211596e-01  3.4062312e+00 -1.1732076e+00 -1.1220042e+00
 -8.3227366e-01  1.7762926e+00  3.2891238e-01  1.1557896e+00
 -1.4838103e+00  1.8187978e+00 -2.4823093e+00 -2.1340424e-01
 -1.2898295e+00  4.1127759e-01 -2.1171091e+00 -1.1878524e+00
 -1.2894152e+00 -5.4592645e-01  1.1809294e-01  3.8364828e-01
 -8.8933170e-01  1.2014093e-01 -5.4570571e-02  3.0548832e-01
 -1.0915042e+00  2.5049356e-01 -7.0697218e-01  7.7669299e-01
 -1.1601051e+00 -1.5831035e-01 -9.2326111e-01  1.3182090e-01]
[[-6.8120978e+02  3.8078697e+01  2.9524112e-01  5.3980947e+00
   9.3655224e+00  2.9750962e+00  2.3212700e+00  2.7702498e+00
  -2.1211596e-01  3.4062312e+00 -1.1732076e+00 -1.1220042e+00
  -8.3227366e-01  1.7762926e+00  3.2891238e-01  1.1557896e+00
  -1.4838103e+00  1.8187978e+00 -2.4823093e+00 -2.1340424e-01
  -1.2898295e+00  4.1127759e-01 -2.1171091e+00 -1.1878524e+00
  -1.2894152e+00 

'up_to_learn'